Exercises:
**E01: train a trigram language model**, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model? 


**E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set**. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?  


**E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model** - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

**E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?**


**E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?**  


**E06: meta-exercise! Think of a fun/interesting exercise and complete it.**

In [2]:
words = open('../names.txt', 'r').read().splitlines()

In [4]:
print(len(words), min(len(w) for w in words), max(len(w) for w in words))

32033 2 15


In [5]:
import torch

ModuleNotFoundError: No module named 'torch'

In [6]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import torch

In [8]:
N = torch.zeros((27,27), dtype=torch.int32)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8560\2082274462.py:1: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:77.)
  N = torch.zeros((27,27), dtype=torch.int32)
